1. Make sure that the single GPU code runs on Polaris.
- Yes. The script runs correctly on a single GPU, completing a 10-epoch sweep in 16.26 s. 

2. The counting of ranks, does not necessarily has to be a mix-and-match between mpi4py and PALS. Try to implement the rank counting method using just PALS or mpi4py. device_count() methods can be useful here.
- We can use the following commands for a pure mpi4py implementation:
```
MPI-only: RANK=MPI.COMM_WORLD.Get_rank()
WORLD_SIZE=MPI.COMM_WORLD.Get_size()
LOCAL_RANK=RANK % torch.cuda.device_count()
```
and the following for a PALS only implementation:
```
RANK=int(os.environ["PALS_RANKID"])
WORLD_SIZE=int(os.environ["PALS_SIZE"])
LOCAL_RANK=int(os.environ["PALS_LOCAL_RANKID"])
```

3. Play with different dimensions of the src and tgt tensors.
- Timing various target lengths with same source lengths shows smooth growth up to tgt=60, then a sharp jump for src=1 at tgt≥70. Below are some configurations I tried:
```
(1,20): 4.57 s;
(1,1): 4.33 s;
(1,50): 4.87 s;
(1,60): 5.33 s;
(60,1): 5.49 s;
(60,60): 7.78 s;
(1,70): 50.47 s;
(1,75): 44.48 s;
(1,100): 45.50 s.
```
![HW1](hw1.jpg)

4. Explore the cost of collective communication, by setting up a scenario, where you have only two ranks, but each rank resides on a different node. Profile and try to reason about the results.
- With two ranks on the same node the run took ~11 s; placing one rank per node increased total time to ~38 s. Profiling shows nccl:all_reduce rising from ~8 ms intra-node to ~199 ms inter-node. The slowdown is communication-bound rather than compute-bound.

5. Try other file formats to explore the I/O bottleneck.
- Replacing HDF5 with a single .pt file reduced average DataLoader step wall time from ~2.8 ms (HDF5) to ~0.96 ms (.pt). 

6. Make the tensors really large, specially the 2nd and 3rd dimension and explore different data types.
- Continuing from the observations from Question 3, we notice that there is a sudden jump in increasing the sequence lengths exposing a data-pipeline bottleneck. Between tgt=60 and 70, total time inflated from 5.33 s to 50.47–63.90 s; profiling attributes the jump to CPU-side batch construction, where the default collate’s aten::stack path incurs heavy copies/allocations at that size, starving the GPU and reducing overlap with NCCL. I tried float16/bfloat16 but it didn't help with this problem.